# TROCS Input Decorrect   

## 20240917
1. TROCS INPUT값 끌어와서 FITTING한 후에 residual_x값에서 빼준걸 저장

# 20240918

1. k값 끌어오는거 주의해야함. 
   ''' 3차까지만 '''
        # Y_dx는 rk1부터 rk71까지 홀수 열을 추출
        Y_dx = rk_values.iloc[:, :20:2].values.flatten()

        # Y_dy는 rk2부터 rk72까지 짝수 열을 추출
        Y_dy = rk_values.iloc[:, 1:20:2].values.flatten()




In [1]:
import pandas as pd
import numpy as np
import os
from openpyxl import load_workbook
import openpyxl



##### TROCS Input Decorrect ##### 

def trocs_decorrect(def_rawdata, def_trocs_input):

    
    # 'shot' 별로 데이터를 그룹화 (고유한 lot_id, die_x, die_y 조합)
    grouped = df_rawdata.groupby(['LOT_ID', 'DieX', 'DieY'])
    
    # 예측 결과를 저장할 리스트
    trocs_input_list = []

    # 각 그룹에 대해 연산 수행
    for (lot_id, diex, diey), group in grouped:

        # 독립변수 설정 ('coordinate_X', 'coordinate_Y'를 독립변수로 사용)
        rx = group['coordinate_X'].values
        ry = group['coordinate_Y'].values

        # 독립 변수 배열구성
        X_dx = np.vstack([
            np.ones(len(rx)),
            (rx)/1e6,     (ry)/1e6, 
            (rx**2)/1e9,  (rx*ry)/1e9,     (ry**2)/1e9,
            (rx**3)/1e12, (rx**2*ry)/1e12, (rx*ry**2)/1e12,    (ry**3)/1e12,
            
        ]).T

        X_dy = np.vstack([
            np.ones(len(ry)), 
            (ry)/1e6,     (rx)/1e6,
            (ry**2)/1e9,  (ry*rx)/1e9,     (rx**2)/1e9,
            (ry**3)/1e12, (ry**2*rx)/1e12, (ry*rx**2)/1e12,    (rx**3)/1e12,
            
        ]).T
        
        

        # 종속변수 설정 ( TROCS INPUT의 RK값을 독립변수로 사용)
        # trocs 시트에서 해당 LOT_ID에 해당하는 rk1~rk72 값을 추출
        trocs_row = df_trocs_input[(df_trocs_input['LOT_ID'] == lot_id) & (df_trocs_input['dCol'] == diex) & (df_trocs_input['dRow'] == diey)]
        if trocs_row.empty:
            print(f"LOT_ID {lot_id}에 해당하는 trocs 데이터가 없습니다.")
            continue

        
        # 홀수 rk 값 (Y_dx)과 짝수 rk 값 (Y_dy) 추출
        # rk1 ~ rk72까지의 열을 rk_values로 가져오기
        rk_values = trocs_row.iloc[:, 5:77]  # rk1 ~ rk72 열을 선택


        ''' 3차까지만 '''

        # Y_dx는 rk1부터 rk71까지 홀수 열을 추출
        Y_dx = rk_values.iloc[:, :20:2].values.flatten()  # rk1, rk3, rk5, ..., rk71까지 선택
        print(Y_dx)

        # Y_dy는 rk2부터 rk72까지 짝수 열을 추출
        Y_dy = rk_values.iloc[:, 1:20:2].values.flatten()  # rk2, rk4, rk6, ..., rk72까지 선택

    
        # 결과 확인
        #print("Y_dx:", Y_dx)
        #print("Y_dy:", Y_dy)
                  
        # 행렬 곱을 통해 예측 값 계산
        trocs_fit_x = X_dx.dot(Y_dx)
        trocs_fit_y = X_dy.dot(Y_dy)
        print(trocs_fit_x)

        residual_x_detrocs = group['residual_x'] - trocs_fit_x
        residual_y_detrocs = group['residual_y'] - trocs_fit_y


        # 결과 저장
        trocs_input_list.append(pd.DataFrame({
            'LOT_ID': lot_id,
            'DieX': diex,
            'DieY': diey,
            'trocs_fit_x': trocs_fit_x,
            'trocs_fit_y': trocs_fit_y,
            'residual_x_detrocs' : residual_x_detrocs,
            'residual_y_detrocs' : residual_y_detrocs
               
        }))
        


    # 결과 병합
    df_result = pd.concat(trocs_input_list, ignore_index=True)
    return df_result



   



################################### TROCS Input Decorrect #####################################################################

# 데이타 불러오기 
df_rawdata = pd.read_excel('output_3차.xlsx', sheet_name='RawData-1')
df_trocs_input = pd.read_excel('output_3차.xlsx', sheet_name='Trocs Input')

# TROCS Input(Shot별 RK값)을 fitting하고 residual_x에서 빼주기 
df_trocs_de = trocs_decorrect(df_rawdata, df_trocs_input)

# mrc_de 결과를 엑셀 파일에 저장
with pd.ExcelWriter('output_3차.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_trocs_de.to_excel(writer, sheet_name='TROCS_decorrect', index=False)




# 엑셀 파일 불러오기
file_path = 'C:/vscode/MODULE/output_3차.xlsx'
excel_file = pd.ExcelFile(file_path)
# 기존 "rawdata-1" 시트 데이터 불러오기
df_rawdata = pd.read_excel(excel_file, sheet_name='RawData-1')
# df_mrc_de 불러오기(mrc fitting, de_mrc 데이터)
df_trocs_de = pd.read_excel(excel_file, sheet_name='TROCS_decorrect')
# 기존 데이터에 df_mrc_de의 열을 새롭게 추가하며 병합
df_combined = pd.concat([df_rawdata, df_trocs_de], axis=1)
# 엑셀 파일의 기존 시트에 덮어쓰기
with pd.ExcelWriter(file_path, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    df_combined.to_excel(writer, sheet_name='RawData-1', index=False)




[ 2.23951051e-04 -1.02227163e-02  6.64955897e-02 -2.54062645e-04
  8.58458560e-06  5.39642730e-04 -3.00599563e-05 -5.95607460e-06
  3.11728140e-06  7.34478264e-05]
[ 1.60514307e-03  1.77632337e-03  1.69431437e-03  9.87285882e-04
  9.04321541e-04  7.45810921e-04  3.50857446e-04  2.24123863e-04
 -9.07832228e-05 -3.36898364e-04]
[-9.93883715e-05 -1.78598351e-02  8.29900006e-03  2.98410290e-04
  4.70984466e-05 -1.22328238e-04  7.65709480e-06  2.59898130e-06
  9.01224400e-07 -1.33284954e-05]
[ 1.44497995e-04 -1.90180536e-04 -3.62977452e-04  4.63736301e-05
 -8.37192396e-05  6.30360520e-05 -7.00266399e-05  3.30003166e-05
 -1.02389329e-04 -1.95231134e-05 -1.56673557e-04 -2.40694991e-04
 -2.10013168e-04 -1.78557919e-04 -3.02961981e-04]
[-4.13364232e-04 -3.94417181e-02 -8.72615015e-03  1.16448094e-05
  4.25514813e-04  1.36407253e-03  3.91643862e-05  1.41538201e-05
  1.90259353e-05 -8.77110967e-05]
[-3.84786010e-04 -3.16104773e-04 -7.98320126e-04  8.83690373e-05
 -4.27853638e-04 -6.25531585e-04 -